In [36]:
!git clone https://github.com/AI4Finance-LLC/FinRL-Library.git

Cloning into 'FinRL-Library'...
remote: Enumerating objects: 2824, done.
remote: Total 2824 (delta 0), reused 0 (delta 0), pack-reused 2824
Receiving objects: 100% (2824/2824), 30.68 MiB | 32.32 MiB/s, done.
Resolving deltas: 100% (1698/1698), done.


In [35]:
pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to /tmp/pip-req-build-_rrrw740
  Running command git clone -q https://github.com/AI4Finance-LLC/FinRL-Library.git /tmp/pip-req-build-_rrrw740
  Created wheel for finrl: filename=finrl-0.3.0-cp37-none-any.whl size=38201 sha256=ff39b87598102736832ca54bc906640f7372906dd3ae8f963caea0fcf608d434
  Stored in directory: /tmp/pip-ephem-wheel-cache-ziepoiop/wheels/9c/19/bf/c644def96612df1ad42c94d5304966797eaa3221dffc5efe0b
Successfully built finrl


In [ ]:
cd /content/FinRL-Library/

/content/FinRL-Library


In [ ]:
pip install -r requirements.txt


In [ ]:
#importing libraies
import pandas as pd
import numpy as np
import matplotlib
import datetime


In [ ]:
from finrl.marketdata.yahoodownloader import YahooDownloader

dataDF = YahooDownloader(start_date = '2020-02-01', end_date = '2021-02-01', ticker_list = ['AAPL']).fetch_data()
dataDF.head(20)

[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (251, 8)


,date,open,high,low,close,volume,tic,day
0,2020-02-03,76.074997,78.372498,75.555000,76.390160,173985600,AAPL,0
1,2020-02-04,78.827499,79.910004,78.407501,78.912079,136616400,AAPL,1
2,2020-02-05,80.879997,81.190002,79.737503,79.555550,118826800,AAPL,2
3,2020-02-06,80.642502,81.305000,80.065002,80.486107,105425600,AAPL,3
4,2020-02-07,80.592499,80.849998,79.500000,79.392090,117684000,AAPL,4
5,2020-02-10,78.544998,80.387497,78.462502,79.769165,109348800,AAPL,0
6,2020-02-11,80.900002,80.974998,79.677498,79.287888,94323200,AAPL,1
7,2020-02-12,80.367500,81.805000,80.367500,81.170799,113730400,AAPL,2
8,2020-02-13,81.047501,81.555000,80.837502,80.592773,94747600,AAPL,3
9,2020-02-14,81.184998,81.495003,80.712502,80.612625,80113600,AAPL,4


In [ ]:
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.config import config

In [ ]:
tech_indicator_list=config.TECHNICAL_INDICATORS_LIST

##  https://github.com/jealous/stockstats
tech_indicator_list=tech_indicator_list+['kdjk','open_2_sma','boll','close_10.0_le_5_c','wr_10','dma','trix']
print(tech_indicator_list)

['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma', 'kdjk', 'open_2_sma', 'boll', 'close_10.0_le_5_c', 'wr_10', 'dma', 'trix']


In [ ]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = tech_indicator_list,
                    use_turbulence=False,
                    user_defined_feature = False)

data_df = fe.preprocess_data(dataDF)
data_df.head()

Successfully added technical indicators


,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,kdjk,open_2_sma,boll,close_10.0_le_5_c,wr_10,dma,trix
0,2020-02-03,76.074997,78.372498,75.555000,76.390160,173985600,AAPL,0,0.000000,81.217652,74.084587,100.000000,66.666667,100.000000,76.390160,76.390160,43.213962,76.074997,76.390160,0.0,70.358115,0.0,0.524675
1,2020-02-04,78.827499,79.910004,78.407501,78.912079,136616400,AAPL,1,0.056582,81.217652,74.084587,100.000000,66.666667,100.000000,77.651119,77.651119,54.504493,77.451248,77.651119,0.0,22.914444,0.0,0.524675
2,2020-02-05,80.879997,81.190002,79.737503,79.555550,118826800,AAPL,2,0.092050,81.631954,74.939904,100.000000,78.577118,100.000000,78.285929,78.285929,60.001208,79.853748,78.285929,0.0,29.005363,0.0,0.435891
3,2020-02-06,80.642502,81.305000,80.065002,80.486107,105425600,AAPL,3,0.139523,82.343780,75.328168,100.000000,79.043281,100.000000,78.835974,78.835974,68.586930,80.761250,78.835974,0.0,14.241625,0.0,0.401333
4,2020-02-07,80.592499,80.849998,79.500000,79.392090,117684000,AAPL,4,0.118849,82.025498,75.868896,77.548879,36.455428,62.968347,78.947197,78.947197,67.968617,80.617500,78.947197,0.0,33.268008,0.0,0.337646


In [ ]:
#test train split
from finrl.preprocessing.data import data_split
train = data_split(data_df, start = '2020-02-01', end = '2021-01-01')
trade = data_split(data_df, start = '2021-01-01', end = '2021-02-01')

In [17]:
from finrl.env.env_stocktrading import StockTradingEnv

stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.TECHNICAL_INDICATORS_LIST)*stock_dimension
print(f"Stock data Dimensions: {stock_dimension}, State Spaces: {state_space}")


env_kwargs = {
    "hmax": 100, 
    "initial_amount": 50_000_000/100, #Since in Indonesia the minimum number of shares per trx is 100, then we scaled the initial amount by dividing it with 100 
    "buy_cost_pct": 0.0019, #IPOT has 0.19% buy cost
    "sell_cost_pct": 0.0029, #IPOT has 0.29% sell cost
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4,
    "print_verbosity":5
    
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

Stock data Dimensions: 1, State Spaces: 11
<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [26]:
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 100000, 
    "buy_cost_pct": 0.001, 
    "sell_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [27]:
#agent

from finrl.model.models import DRLAgent

agent = DRLAgent(env = env_train)

In [28]:
#DDPG
DDPG_PARAMS = {"batch_size": 64, "buffer_size": 500000, "learning_rate": 0.0001}
model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=10000)

{'batch_size': 64, 'buffer_size': 500000, 'learning_rate': 0.0001}
Using cuda device
Logging to tensorboard_log/ddpg/ddpg_3
day: 231, episode: 50
begin_total_asset: 500000.00
end_total_asset: 930111.81
total_reward: 430111.81
total_cost: 948.16
total_trades: 231
Sharpe: 2.094
----------------------------------
| environment/        |          |
|    portfolio_value  | 9.3e+05  |
|    total_cost       | 948      |
|    total_reward     | 4.3e+05  |
|    total_reward_pct | 86       |
|    total_trades     | 231      |
| time/               |          |
|    episodes         | 4        |
|    fps              | 148      |
|    time_elapsed     | 6        |
|    total timesteps  | 928      |
| train/              |          |
|    actor_loss       | 2.76e+03 |
|    critic_loss      | 3.44e+03 |
|    learning_rate    | 0.0001   |
|    n_updates        | 696      |
----------------------------------
----------------------------------
| environment/        |          |
|    portfolio_value  |

In [29]:
trade.head()

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,kdjk,open_2_sma,boll,close_10.0_le_5_c,wr_10,dma,trix
0,2021-01-04,133.520004,133.610001,126.760002,129.216919,143301900,AAPL,0,3.457238,137.215735,118.508134,56.612402,64.541763,8.436327,124.695998,120.630709,59.018892,133.800003,127.861934,0.0,62.405975,10.452674,0.475664
1,2021-01-05,128.889999,131.740005,128.429993,130.814514,97664900,AAPL,1,3.229626,137.324537,119.274024,57.830875,65.431303,8.436327,125.107716,120.900938,50.580384,131.205002,128.299280,0.0,51.991402,10.578299,0.457813
2,2021-01-06,127.720001,131.050003,126.379997,126.411102,155088000,AAPL,2,2.663223,137.239446,119.643689,53.543397,31.629837,0.539157,125.415922,121.064551,33.803804,128.305000,128.441568,0.0,99.749355,10.180731,0.421860
3,2021-01-07,128.360001,131.630005,127.860001,130.724655,109578200,AAPL,3,2.533211,137.385133,120.151026,56.789765,55.560894,2.682674,125.984073,121.176621,34.205650,128.040001,128.768080,0.0,64.990658,9.763990,0.388448
4,2021-01-08,132.429993,132.630005,130.229996,131.852966,105158200,AAPL,4,2.492489,137.301279,121.260348,57.591662,72.499882,6.400448,126.545900,121.362320,37.504199,130.394997,129.280814,0.0,55.898704,9.560277,0.360550


In [67]:
    def DRL_prediction(model,name,trade_data, trade_env,trade_obs):
        ### make a prediction based on trained model###
        for i in range(len(trade_data.index.unique())):
            action, _states = model.predict(trade_obs)
            trade_obs, rewards, dones, info = trade_env.step(action)
            if i == (len(trade_data.index.unique()) - 2):
                # print(env_test.render())
                last_state = trade_env.render()

        #df_last_state = pd.DataFrame({'last_state': last_state})
        #print(df_last_state)
        #df_last_state.to_csv('results/last_state_{}_{}.csv'.format(name, i), index=False)
        return last_state

In [68]:
trade = data_split(data_df, start = '2021-01-01', end = '2021-02-01')
e_trade_gym = StockTradingEnv(df = trade, **env_kwargs)
env_trade, obs_trade = e_trade_gym.get_sb_env()

df_account_value = DRL_prediction(model=model_ddpg,name = "DDPG", trade_data = trade, trade_env = env_trade, trade_obs = obs_trade)


In [56]:
df_account_value

[110.0279333496128,
 131.7631072998047,
 770.0,
 2.9013537899216146,
 143.40275784220088,
 122.23929156575811,
 53.397006347906526,
 9.862308391732444,
 1.2243363700441778,
 132.19845479329427,
 125.73289591471354]

In [30]:
from finrl.trade.backtest import backtest_stats

In [69]:
print("==============Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value= df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)

==============Results===========


TypeError: ignored

In [70]:
print("==============Compare to IHSG===========")
%matplotlib inline
backtest_plot(df_account_value, 
             baseline_ticker = '^JKSE', 
             baseline_start = df_account_value.loc[0,'date'],
             baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])

==============Compare to IHSG===========


NameError: ignored